In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import torch
from data import get_midi_data, MidiLoader
from handcrafted import HandcraftedKeyModel
import matplotlib.pyplot as plt
from torch.autograd import Variable

handcrafted = HandcraftedKeyModel()
midi_data = get_midi_data(handcrafted)
midi_loader = MidiLoader(midi_data, batch_size=1, shuffle=True)

In [2]:
dataiter = iter(midi_loader)
midi_vectors, midi_labels = dataiter.next()

In [3]:
print midi_vectors, midi_labels


   44   128    40  ...    128   128   128
[torch.LongTensor of size 1x81676]
 Variable containing:
(  0  ,  0  ,.,.) = 
  0.0000  0.0000  0.0000  ...   0.0000  0.0000  0.0000
  0.0000  0.0000  0.0000  ...   0.0000  0.0000  0.0000
  0.0000  0.0000  0.0000  ...   0.0000  0.2000  0.0000
           ...             ⋱             ...          
  0.0000  0.0000  0.0000  ...   0.0000  0.0000  0.0000
  0.0000  0.0000  0.0000  ...   0.0000  0.0000  0.0000
  0.0000  0.0000  0.0000  ...   0.0000  0.0000  0.7000

(  0  ,  1  ,.,.) = 
  0.0000  0.0000  0.0000  ...   0.0000  0.0000  0.0000
  0.0000  0.0000  0.0000  ...   0.0000  0.0000  0.0000
  0.0000  0.0000  0.0000  ...   0.0000  0.2000  0.0000
           ...             ⋱             ...          
  0.0000  0.0000  0.0000  ...   0.0000  0.0000  0.0000
  0.0000  0.0000  0.0000  ...   0.0000  0.0000  0.0000
  0.0000  0.0000  0.0000  ...   0.0000  0.0000  0.7000

(  0  ,  2  ,.,.) = 
  0.0000  0.0000  0.0000  ...   0.0000  0.0000  0.0000
  0.1000  

In [19]:
test = torch.Tensor([[1., 2., 3.], [4., 1., 2.5]])
sorted_t, sorted_i = torch.sort(test.view(-1))
out_test = test.view(-1)
out_test.zero_()
out_test[sorted_i[-3:]] = 1
print test.view(2, 3)


 0  0  1
 1  0  1
[torch.FloatTensor of size 2x3]

